# AIP 4 - Project Rechin
## Strategic Thinking - CA3 - Sprint 2
## Project Rechin - LSTM for multivariate time series prediction 

In [ ]:
# Sent for phase 3

# Notes for further development

## Source code for BasicLSTMCell in tensorflow

https://github.com/tensorflow/tensorflow/blob/ef96faaf02be54b7eb5945244c881126a4d38761/tensorflow/python/ops/rnn_cell.py#L275

***

## On num_units

https://stackoverflow.com/questions/37901047/what-is-num-units-in-tensorflow-basiclstmcell

https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/
    
num_units can be interpreted as the analogy of hidden layer from the feed forward neural network.

The number of nodes in hidden layer of a feed forward neural network is equivalent to num_units number of LSTM units in a LSTM cell at every time step of the network.

***

## Model configuration: customisation posibilities

***

## LSTM Layer parameters and defaults

***

## Library imports

In [1]:
# Data acquisition
import pickle
import mysql.connector 
import os

# Data analysis basics
import numpy as np
import pandas as pd
import datetime

#EDA
#import pandas_profiling # https://anaconda.org/conda-forge/pandas-profiling # conda install ipywidgets

#Machine Learning
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

# ML > LSTM 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input

# ML > Save model
from tensorflow.keras.models import model_from_json

# ML > Optimizer > Adam just for trying
from tensorflow.keras.optimizers import Adam

# ML > Metrics
from tensorflow.keras import metrics as metrics_tf
import sklearn.metrics as metrics

# ML > GPU for tensorflow
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# Feature Scaling
from sklearn.preprocessing import StandardScaler

#Viz
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
# Set plot size 
from pylab import rcParams
%matplotlib inline

#Cosmetic: Disabling notebook warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Check If GPU or CPU

GPU runs much faster for model training than CPU but you must install CUDA first. 

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


***

# Data acquisition and cleaning

## Get Master Dataframe
For now we are using a pickled dataframe until we can get our model sorted out then we will get the data from the  AWS database that we have inputed and can retreive the data needed.

In [4]:
class Stock():
    def __init__(self, symbol):
        self.symbol = symbol
        self.data = []
        self.indicators = []
        
    def getStockData(self, source, user):
        '''This function gets the stock data either from the local computer or online. Returns a dataframe of the 
        one minute stock data.

        Args:
        symbol (string): this is the symbol of the data that is needed.
        source (string): This string is for either online or local hard drive access of stock one minute data.
        user (string): This is the number in the future you want to predict out.

        Returns:
        df (dataframe): stock data by the symbol requested in dataframe sorted by date.  Indicators will be added later.

        Notes: 
        '''
        if source == 'local':
            if user == 'antonio':
                df = pickle.load(open( "D:\Rechin_CCT\Rechin\data\masterDf_Phase2.p", "rb" ))
                df = df[df.Symbol == symbol]
                # Make sure the data is sorted ascending order by datetime.
                df.sort_values(by=['date'], inplace=True)
                df.reset_index(inplace=True, drop=True)
                df.head()
            elif user == 'james':
                df = pickle.load(open( "D:\Downloads\masterDf_Phase2.p", "rb" ))
                df = df[df.Symbol == symbol]
                # Make sure the data is sorted ascending order by datetime.
                df.sort_values(by=['date'], inplace=True)
                df.reset_index(inplace=True, drop=True)
                df.head()
            else:
                print('User not recognized!')
                df = pd.DataFrame()
        elif source == 'cloud':
            # Credentials to database connection
            hostname="database-rechin.123456.eu-west-1.rds.amazonaws.com" #Antonio's AWS
            uname="admin"
            pwd="rechin"
            dbName = 'rechindbcct'
            
            # Connect to Specific Database
            db = mysql.connector.connect(host= hostname,
              user=uname, password=pwd, database = dbName)
            
            # Get all symbols from database
            cursor = db.cursor()
            cursor.execute("""SELECT * FROM rechin_minute_price WHERE symbol = (%s)""", (self.symbol,))
            result = cursor.fetchall()
            
            # Create dataframe for symbol
            columns = [i[0] for i in cursor.description]
            df = pd.DataFrame(result, columns=columns)
        df = df.drop(columns=['data_vendor','created_date', 'last_updated'])
        df.sort_values(by="date", inplace=True)
        df.reset_index(inplace=True, drop=True)
        self.data = df
        return df
    
    def getTechnicalIndicators(self, TAtype):
        '''This function adds requested technical indicators to the dataframe to use in features later.

        Args:
        TAtypet (array): This array is the indicators requested with settings.

        Returns:
        df (dataframe): adds techincal indicators to the stock history dataframe.

        Notes: 
        '''
        # use self.data for historical data to do indicators
        # return self.indicators
        pass

## Now time to clean our dataset
There are a few things needed to be done here.  We need to drop columns and deal with some 0 values in volume.  From there the dataset is ready to add technical indicators. 

## Now Put our Label into the Last Column of DataFrame
The preceding columns are features and the right column will will be the label column.  This is the column we will tell the machine we want it to predict.

# Data transformations for ML

## Split the DataSet into Train and Test
Based on the split amount we selected above we will split the dataset into two, Train and Test.  Decided to split the dataset this way rather than using the sklearn split function as it shuffles the dataset and we cannot do that in a time series.

## Scale the Dataset for the Machine to Predict
Scaling makes it easier for the machine to predict as it deals with outliers and puts all within a -1 to 1 range.

## Methods to Reshape the dataset and Later Model Evaluation and Predictions
These methods will be used to put into an array the lookback periods into each row removing the first lookback rows.  Also it deletes the prediction rows at the end of the prediction data. It will also turn the data into arrays for us in the model.

In [5]:
class dataTransformer():
    def __init__(self, scaler, data):
        self.scaler = scaler
        self.data = data
    ## Now time to clean our dataset
    #There are a few things needed to be done here.  
    #We need to drop columns and deal with some 0 values in volume.  
    #From there the dataset is ready to add technical indicators.
    def CleanImpute(self, data, impute):
        '''This function replaces missing volume values in the dataset with the rolling volume average.  
        If imputed is true the missing volume values is replaced volume average.  Columns are dropped 
        that are not needed and index is reset.
        If impute is false the column not needed is dropped and the index is reset.

        Args:
        data (dataFrame): This is the data that will be cleaned with imputed data.

        Returns:
        df (dataframe): Cleaned dataframe with either imputed data and removed columns or just removed colomns
        depending if imputed is true or false.

        Notes: 
        '''
        df = data.copy()
        if impute:
            # Clean dirty data
            ### Imputing 20K zeroes with rolling average for the last 360 minutes
            data["volume_mean"]=data["volume"].replace(0, np.nan)
            data["volume_mean"] = data["volume_mean"].fillna(data["volume_mean"].rolling(360,min_periods=1).mean())

            # Now that dataset has imputed values we make our final dataframe to use for our model.
            df = data.copy()
            
            # replace column with missing values with imputed column.
            df['volume'] = df['volume_mean']

            # Now drop columns that will not be needed for our model.
            df = df.drop(columns=['volume_mean'])
            df = df.drop(columns=['Symbol'])
            df.reset_index(inplace=True, drop=True)
        else:
            df = df.drop(columns=['Symbol'])
            df.reset_index(inplace=True, drop=True)

        return df

    def Index_ReorderIndependent(self, df):
        '''This function puts the label into the last column of the dataframe.  
        The preceding columns are features and the right column will will be the label column.  
        This is the column we will tell the machine we want it to predict.

        Args:
        data (dataFrame): This is the data that will be reorderd into label last column.

        Returns:
        df (dataframe): Reordered dataframe with features on rows 0 to second to last and last row is the
        label or target value.

        Notes: 
        We need to add to this function which is the label and do this programatically.  
        '''
        df.set_index(pd.DatetimeIndex(df["date"]), inplace=True)
        df.drop(columns=['date'],inplace=True)
        first_column = df.pop('close')
        # insert column using insert(position,column_name, first_column) function
        df.insert(len(df.columns), 'close', first_column)
        return df
    
    def GetTrainTest(self, df, train_split_percentage):
        '''This function takes our dataset and splits it into train dataset and test dataset.  This will 
        help our machine learn and then test the model to validate the model.

        Args:
        data (dataFrame): This is the data that you will split into train and test.
        train_split_percentage (float): This is percentage you want to split the dataset by.

        Returns:
        train (array): dataset to use for the training of the model.
        test (array): dataset to validate the model training.

        Notes: 
        '''
        # Get the index where we will split the dataset
        splitIndex = int(round(len(df) * train_split_percentage,0))

        # Split the dataset into two
        train = df.iloc[:splitIndex,:]
        test = df.iloc[splitIndex+1:,:]
        return train, test
    
    def scaleData(self, train, test):
        '''This function scales the dataset to make easier for the LSTM model to train and predict.  This deals
        with outliers.

        Args:
        train (array): dataset to scale for the training of the model.
        test (array): dataset to scale for the testing the model.

        Returns:
        train (array): scaled dataset to use for the training of the model.
        test (array): scaled dataset to validate the model training.
        scaler (scaler model): this model will be used later to inverse transform the predicted label.

        Notes: 
        will add MinMaxScaler to try another scaler and see if this improves accuracy and avoids overfitting.
        '''
        if self.scaler == 'StandardScaler':
            # Call the scaler function
            scaler = StandardScaler()
            # Scale the Train & Test Data with same fitting (scaling factor)
            scaler = scaler.fit(train)
            scaled_train = scaler.transform(train)
            scaled_test = scaler.transform(test)
        elif self.scaler == 'MinMaxScaler':
            # Need to put in minmaxscaler
            pass
        return scaled_train, scaled_test, scaler
    
    # Google style documentation on functions:
    #https://google.github.io/styleguide/pyguide.html
    def reformatDataset(self, n_past, n_future, data):
        '''This function takes the dataset input and returns the data in a format usable by the LSTM Model.  
        The dataset is reshaped to the number of past minutes for each iteration and then 
        gets the predicted minutes.  At the end of the function the data is converted into an array.

        Args:
        n_past (int): This is the number of past observations to go back to use in each row for the LSTM.
        n_future (int): This is the number in the future you want to predict out.
        data (dataFrame): This is the data that you will reshape into features and label.

        Returns:
        features (array): features in shape ready for LSTM training or LSTM predictions.
        label (array): label ready for LSTM training or LSTM predictions.

        Notes: 
        https://github.com/bnsreenu/python_for_microscopists/blob/master/181_multivariate_timeseries_LSTM_GE.py
        '''
        # Create empty arrays to input the formated data
        xDataset = []
        yDataset = []
        # Loop through the dataset and append both the feature and label array in preparation for LSTM model.
        for i in range(n_past, len(data) - n_future + 1):
            xDataset.append(data[i - n_past:i, 0:data.shape[1]])
            yDataset.append(data[i + n_future - 1:i + n_future, 0])

        # Convert the data into numpy arrays.
        xDataset, yDataset = np.array(xDataset), np.array(yDataset)
        return xDataset, yDataset

***

# Now Create our RNN LSTM Model and Print Summary
This is model that we have created over various, books, online articles, papers and youtube videos... countless hours.

## Notes on Early Stopping



* **EarlyStopping**: Stop training when a monitored mertric has stopped omproving.

* **<font color=red>monitor</font>**: quantity to be monitored

* **<font color=red>min_delta</font>**: minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta will count as no improvement.

* **<font color=red>patience</font>**: numbers of epochs with no improvement after which training will be stopped.

* **ReduceLROnPLateau**: Reduce learning rate when a metric has stopped improving.
* **<font color=red>factor</font>**: factor by which the learning rate will be reduced. <font color=red>new_lr = lr * factor</font>

### Notes on size of n_units

n_neurons = round(self.X_train.shape[1] / (neuron_scale_factor*(self.X_train.shape[2]+1)))


$
N_{h} = \frac{N_{s}}{(\alpha*(N_{i}+N_{o}))}
$

Ni = number of input neurons.

No = number of output neurons.

Ns = number of samples in training data set.

α = an arbitrary scaling factor usually 2-10.

Source: https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm


In [6]:
class Trainer:
    def __init__(self, loss, optimizer, epochs, X_train, y_train, X_test, y_test, activation, 
            layerNumber, modelName, patience, customBatchSize, neuron_scale_factor, split):
        self.modelName = modelName
        self.layerNumber = layerNumber
        self.model = Sequential()
        self.loss = loss
        self.optimizer = optimizer
        self.epochs = epochs
        self.X_train = X_train
        self.y_train = y_train
        self.activation = activation
        self.patience = patience
        self.customBatchSize = customBatchSize
        self.logs_save_path = "logs/fit/"
        self.model_save_path = "logs/model/"
        self.neuron_scale_factor = neuron_scale_factor
        self.split = split
        
    def createModel(self):
        '''This function creates our model based on a number of input arguements.  Firstly we want to clear
        any prior sessions to get a good clean model.  Number of Nuerons is also processed here based on the
        size of the dataset and features.  Then build the number of layers based on the inputs.  Finally,
        compile the model and print a summary of the model.

        Args:
        loss (string): This string is the model which loss function used in the model's trainging and test.
        optimizer (string): This string is the optimization function used in the model's trainging and test.
        epochs (int): This is int is the number of epochs to use in traning the model.
        X_train (array): This data is the feature data will be used to train the model.
        y_train (array): This data is the label data will be used to train the model.
        X_test (array): This data is the feature data will be used to test the model.
        y_test (array): This data is the label data will be used to test the model.
        activation (string): This is the function that is required between matrix.
        layerNumber (int): How many layers to put into the model.
        modelName (string): The name of the model to save the model and loss functions.
        patience (int): The max number of times the epochs can go without improvement in the loss function.
        customBatchSize (int): Batch sizes of the data to use in the model.

        Returns:
        model (array): features in shape ready for LSTM training or LSTM predictions.
        n_nuerons (array): label ready for LSTM training or LSTM predictions.

        Notes: 

        '''
        #####################################
        #Reset the backend for the next iteration (rerun the model)
        tf.keras.backend.clear_session()
        #####################################

        # Initialize the model
        model = self.model

        # Model with n_neurons = inputshape Timestamps, each with x_train.shape[2] variables
        #n_neurons = self.X_train.shape[1] * self.X_train.shape[2]
    
        n_neurons = round(self.X_train.shape[1] / (self.neuron_scale_factor*(self.X_train.shape[2]+1)))
        
        layerNumber = self.layerNumber+3
        
        print('Number of look back minutes: {}\nNumber of features: {}\nTrainTest split: {}\nBatches: {}\nNumber of Neurons: {}\nLayers: {}'.
              format(self.X_train.shape[1], self.X_train.shape[2], self.split, self.customBatchSize, n_neurons, layerNumber))

        # These two layers work but not sure if it is a proper LSTM
        model.add(LSTM(units=n_neurons, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]),
                       return_sequences=True, name=self.modelName + "_Input"))

        if self.layerNumber > 1:
            for layer in range(self.layerNumber):
                    self.layerName = self.modelName+"_"+str(layer)
                    if layer < self.layerNumber:
                        model.add(LSTM(units=n_neurons, activation=self.activation, 
                                       return_sequences=True, name =  self.layerName))
                    elif layer == self.layerNumber:
                        self.layerName = self.modelName+"_"+str(layer)
                        model.add(LSTM(units=n_neurons, activation=self.activation, 
                                       return_sequences=False, name =  self.layerName))

        model.add(Dropout(0.20))
        model.add(Dense(y_train.shape[1]))

        model.compile(optimizer=self.optimizer, loss=self.loss,
                      metrics=[metrics_tf.mean_squared_error,
                               metrics_tf.mean_absolute_error, 
                               metrics_tf.mean_absolute_percentage_error])

        model.summary()
        return model, n_neurons
    
    def trainModel(self):
        '''This function takes model and add additional callbacks into the model.  Here the model is also
        saved along with the logs of the model during the training process. The model is saved and if a better
        model comes along it is saved and overwrites the model.  This only happens on each model and does not
        overwrite other models.  Patience is added to the callbacks to stop the training early if no improvement.
        Validation data is added to the model to validate the trained model against test data.

        Args:
        They are brought in when the class is called an brought into this function.

        Returns:
        history (model): returns a model after training through the number of epochs.

        Notes: 
        '''
        es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=self.patience, verbose=1)
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=self.patience, verbose=1)
        #https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
        modelName = self.modelName +'.h5'
        saveName = os.path.join(self.model_save_path, modelName)
        mcp = ModelCheckpoint(filepath=saveName, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
        
        # Save logs of callbacks
        logdir=os.path.join(self.logs_save_path, (datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+self.modelName))
        tb = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
        
        # fit the model
        history = self.model.fit(self.X_train, self.y_train, epochs=self.epochs, batch_size=self.customBatchSize, 
                            validation_split=0.1, verbose=1, callbacks=[es, rlr, mcp, tb], 
                            validation_data=(X_test, y_test))

        return history

## User Variables

Here is the inital variables that are needed later in selecting our stock to use, spliting the data and the discrete time inputs.

In [7]:
# Stock symbol we will be using to train our model
symbol = "MNST"
# Past look back minutes to train and amount of futrue minutes to predict
lookback_minutes = 360
prediction_minutes = 10 # AKA timesteps

## LSTM Model parameters

In [8]:
# Split amount for train and test
#train_split " : [0.50, 0.70, 0.90]
#ambitious train_split = [0.50, 0.60, 0.70, 0.80, 0.90]

#50
LSTM_1_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_2_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_3_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}
#70
LSTM_4_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_5_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_6_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#90
LSTM_7_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_8_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_9_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#70_4_Layers
LSTM_10_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_11_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_12_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#70_5_Layers
LSTM_13_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_14_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_15_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_dict_list = [LSTM_1_dict,LSTM_2_dict,LSTM_3_dict,LSTM_4_dict,LSTM_5_dict,LSTM_6_dict,
                  LSTM_7_dict,LSTM_8_dict,LSTM_9_dict,LSTM_10_dict,LSTM_11_dict,LSTM_12_dict,
                  LSTM_13_dict, LSTM_14_dict, LSTM_15_dict ]

 ## Function Calling

In [9]:
df = Stock(symbol=symbol).getStockData(source='local',user='james')
df

,Symbol,date,open,high,low,close,volume
0,MNST,2020-10-15 09:30:00-04:00,81.830,82.495,81.740,81.900,1701.0
1,MNST,2020-10-15 09:31:00-04:00,81.900,81.900,81.900,81.900,0.0
2,MNST,2020-10-15 09:32:00-04:00,81.710,81.720,81.710,81.720,105.0
3,MNST,2020-10-15 09:33:00-04:00,81.990,82.005,81.990,82.005,15.0
4,MNST,2020-10-15 09:34:00-04:00,81.880,81.950,81.870,81.950,300.0
...,...,...,...,...,...,...,...
98136,MNST,2021-10-15 15:56:00-04:00,85.570,85.575,85.535,85.555,500.0
98137,MNST,2021-10-15 15:57:00-04:00,85.575,85.640,85.575,85.630,1063.0
98138,MNST,2021-10-15 15:58:00-04:00,85.610,85.610,85.590,85.605,510.0
98139,MNST,2021-10-15 15:59:00-04:00,85.610,85.610,85.560,85.575,502.0


In [11]:
transform = dataTransformer(scaler='StandardScaler', data=df)

In [12]:
df = transform.CleanImpute(data=df, impute=False)
df

,date,open,high,low,close,volume
0,2020-10-15 09:30:00-04:00,81.830,82.495,81.740,81.900,1701.0
1,2020-10-15 09:31:00-04:00,81.900,81.900,81.900,81.900,0.0
2,2020-10-15 09:32:00-04:00,81.710,81.720,81.710,81.720,105.0
3,2020-10-15 09:33:00-04:00,81.990,82.005,81.990,82.005,15.0
4,2020-10-15 09:34:00-04:00,81.880,81.950,81.870,81.950,300.0
...,...,...,...,...,...,...
98136,2021-10-15 15:56:00-04:00,85.570,85.575,85.535,85.555,500.0
98137,2021-10-15 15:57:00-04:00,85.575,85.640,85.575,85.630,1063.0
98138,2021-10-15 15:58:00-04:00,85.610,85.610,85.590,85.605,510.0
98139,2021-10-15 15:59:00-04:00,85.610,85.610,85.560,85.575,502.0


In [13]:
df = transform.Index_ReorderIndependent(df)
df

,open,high,low,volume,close
date,,,,,
2020-10-15 09:30:00-04:00,81.830,82.495,81.740,1701.0,81.900
2020-10-15 09:31:00-04:00,81.900,81.900,81.900,0.0,81.900
2020-10-15 09:32:00-04:00,81.710,81.720,81.710,105.0,81.720
2020-10-15 09:33:00-04:00,81.990,82.005,81.990,15.0,82.005
2020-10-15 09:34:00-04:00,81.880,81.950,81.870,300.0,81.950
...,...,...,...,...,...
2021-10-15 15:56:00-04:00,85.570,85.575,85.535,500.0,85.555
2021-10-15 15:57:00-04:00,85.575,85.640,85.575,1063.0,85.630
2021-10-15 15:58:00-04:00,85.610,85.610,85.590,510.0,85.605


In [14]:
n_features = len(df.columns)

In [15]:
# Loop different models
for LSTM_dict in LSTM_dict_list:
    # Split the data based on split size
    train, test = transform.GetTrainTest(df, LSTM_dict["train_split"])
    # Scale the data
    scaled_train, scaled_test, scaler = transform.scaleData(train, test)
    
    # Transform our dataset into features and label variable and reshaped 3D 
    # First the Train Dataset
    X_train, y_train = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_train)
    # Second the Test Dataset
    X_test, y_test = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_test)
    
    #modelName = 'LSTM_model_layers_' + str(LSTMLayers+2) + '_train_split_' + str(split)
    modelName = 'LSTM_' + str(LSTM_dict["LSTMLayers"]+3) + '_layers_tS_' + str(LSTM_dict["train_split"])
                                                            + '_bs_'+str(LSTM_dict["custom_batch_size"])
    
    # Create the model
    model1 = Trainer(loss ='mse', optimizer='adam', epochs=LSTM_dict["custom_epochs"], X_train=X_train, y_train=y_train, 
                 X_test=X_test, y_test=y_test, activation='tanh', layerNumber=LSTM_dict["LSTMLayers"], modelName=modelName, 
                 patience=LSTM_dict["custom_patience"], customBatchSize=LSTM_dict["custom_batch_size"], 
                    neuron_scale_factor = LSTM_dict["neuronScaleFactor"], split = LSTM_dict["train_split"])
    model, n_neurons = model1.createModel()
    
    #Fit the model
    history = model1.trainModel()

Number of look back minutes: 360
Number of features: 5
TrainTest split: 0.5
Batches: 64
Number of Neurons: 30
Layers: 6
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_6_layers_tS_0.5_bs_64_I (None, 360, 30)           4320      
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_0 (None, 360, 30)           7320      
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_1 (None, 360, 30)           7320      
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_2 (None, 360, 30)           7320      
_________________________________________________________________
dropout (Dropout)            (None, 360, 30)           0         
_________________________________________________________________
dense (Dense)                (None, 360, 1)            31        
To

685/685 [==============================] - 47s 68ms/step - loss: 0.0442 - mean_squared_error: 0.0442 - mean_absolute_error: 0.1568 - mean_absolute_percentage_error: 182.1885 - val_loss: 0.1532 - val_mean_squared_error: 0.1532 - val_mean_absolute_error: 0.3340 - val_mean_absolute_percentage_error: 20.74770464 - mean_squared_error: 0.0464 - mean_absolute_error: 0.1591 - mean_absolute_percentage_error: 18 - ETA: 43s - loss: 0.0460 - mean_squared_error: 0 - ETA: 40s - loss: 0.0452 - mean_squared_error: 0.0452 - mean_absolute_error: 0.1584 - mean_absolute_percenta - ETA: 39s - loss: 0.0451 - mean_squared_error: 0.0451 - mean_absolute_er - ETA: 38s - loss: 0.0449 - mean_squared_error: 0.0449 - mean_absolute_error: 0.1581 - mean_a - ETA: 37s - loss: 0.0448 - mean_squared_error: 0.0448 - mean_a - ETA: 35s - loss: 0.0447 - mean_squared_error: 0.0447 - mean_absolute_error: 0.1 - ETA: 33s - loss: 0.0446 - mean_squared_error: 0.0446 - mean_absolute_error: 0.1 - ETA: 32s - loss: 0.0446 - mean_squar

685/685 [==============================] - 47s 69ms/step - loss: 0.0415 - mean_squared_error: 0.0415 - mean_absolute_error: 0.1522 - mean_absolute_percentage_error: 177.0124 - val_loss: 0.1717 - val_mean_squared_error: 0.1717 - val_mean_absolute_error: 0.3574 - val_mean_absolute_percentage_error: 22.3208absolute_error: 0.1530 - mean_absolute_percentage_error: 111. - ETA: 41s - loss: 0.0416 - mean_squared_error: 0.0416 - mean_absolute_error: 0.1530 - mean_absolut - ETA: 40s - loss: 0.0417 - mean_squared_error: 0.0417 - mean_absolute_error: 0.153 - ETA: 40s - lo - ETA: 36s - loss: 0.0413 - mean_squared_error: 0.0413 - mean - ETA: 34s - loss: 0.0412 - mean_sq - ETA: 31s - loss: 0.0412 - mean_squared_error: 0.0412 - mean_absolute_error: 0.1518 - mean_absol - ETA: 30s - loss: 0.0412 - mean_squared_error: 0.0412 - mean_absolute_error: 0.1517 - mean_absolute_percentage_error - ETA: 29s - loss: 0.0412 - mean_squared_error: 0.0412 - mean_absolute_error: 0.1517 - mean_absolute_percentage_error: 

685/685 [==============================] - 48s 70ms/step - loss: 0.0409 - mean_squared_error: 0.0409 - mean_absolute_error: 0.1522 - mean_absolute_percentage_error: 171.5944 - val_loss: 0.1333 - val_mean_squared_error: 0.1333 - val_mean_absolute_error: 0.2973 - val_mean_absolute_percentage_error: 18.4639ETA: 40s - loss: 0.0396 - mean_squared_error: 0.0396 - mean_absolute_error: 0.1509 - mean_absolute_percentage_e - ETA: 40s - loss: 0.0396 - mean_squared_error: 0.0396 - mean_absolute_error: 0.150 - ETA: 38s - loss: 0.0397 - mean_squared_error: 0.0397 - mean_absolute_error: 0.1510 - mean_absol - ETA: 38s - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.1511 - mean_absolute_percentage_error: 21 - ETA: 37s - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.1511 - mean_absolute_percentage_error: 217. - ETA: 37s - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.1512 - mean_absolute_percenta - ETA: 37s - loss: 0.0399 - mean_squared_erro

685/685 [==============================] - 47s 69ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - mean_absolute_error: 0.1487 - mean_absolute_percentage_error: 145.6338 - val_loss: 0.1481 - val_mean_squared_error: 0.1481 - val_mean_absolute_error: 0.3150 - val_mean_absolute_percentage_error: 19.3686- mean_squared_error: 0.0382 - mean_absolute_error: 0.1462 - mean_absolute_percentage - ETA: 41s - loss: 0.0385 - mean_squared_error: 0.0385 - mean_absolute_error: 0.1468 - mean_absolute_percenta - ETA: 40s - loss: 0.0387 - mean_squared_error: 0.0387 - mean_absolute_error: 0.1471 - mean_absolute_perc - ETA: 40s - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.1474 - mean_absolut - ETA: 39s - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.1476 - mean_absolute_percenta - ETA: 38s - loss: 0.0390 - mean_squared_error: 0.0390 - mean_absolute_error: 0.1477 - mean_absolute_percentage_error: 112.97 - ETA: 38s - loss: 0.0390 - mean_squared_error: 0.0390 - 

...

961/961 [==============================] - 106s 110ms/step - loss: 0.0422 - mean_squared_error: 0.0422 - mean_absolute_error: 0.1517 - mean_absolute_percentage_error: 174.9403 - val_loss: 0.0139 - val_mean_squared_error: 0.0139 - val_mean_absolute_error: 0.0915 - val_mean_absolute_percentage_error: 45.1807ared_error: 0.0444 - mean_absolute_error: 0.1561 - mean_absolute_percentage_error: 193.1 - ETA: 1:33 - loss: 0.0443 - mean_squared_error: 0.0443 - mean_absolute_error: 0.1560 - mean_absolute_percentage_error:  - ETA: 1:32 - loss: 0.0442 - mean_squared_error: 0.0442 - mean_absolute_e - ETA: 1:26 - loss: 0.0437 - mean_squared_error: 0.0437 - mean_absolute_error: 0.1547 - mean - ETA: 1:22 - loss: 0.0434 - mean_squared_error: 0.0434 - mean_absolute_error: 0.1542 -  - ETA: 1:18 - loss: 0.0432 - mean_squared_error: 0.0432 - mean_absolute_error: 0.1536 - mean_absolute_per - ETA: 1:16 - loss: 0.0431 - mean_squared_error: 0.0431 - mean_absolute_error: 0.1534 - mean_ - ETA: 1:12 - loss: 0.0429 

961/961 [==============================] - 100s 104ms/step - loss: 0.0395 - mean_squared_error: 0.0395 - mean_absolute_error: 0.1467 - mean_absolute_percentage_error: 156.5488 - val_loss: 0.0135 - val_mean_squared_error: 0.0135 - val_mean_absolute_error: 0.0909 - val_mean_absolute_percentage_error: 49.7586ss: 0.0401 - mean_squared_error: 0.0401 - mean_absolute_error: 0.1477 - mean_absolute_p - ETA: 1:23 - loss: 0.0399 - mean_squared_error: 0.0399 - mean_absolute_error: 0.1474 - mean_absolute_percentage_erro - ETA: 1:22 - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.1472 - mean_absolute_percentage_error: 10 - ETA: 1:21 - loss: 0.0397 - mean_squared_error: 0.0397 - mean_absolute_error: 0.1471 - mean_absolute_percentage_error: 10 - ETA: 1:21 - loss: 0.0397 - mean_squared_error: 0.0397 - mean_absolute_error: 0.1471 - mean - ETA: 1:17 - loss: 0.0395 - mean_squared_error: 0.0395 - mean_absolute_error: 0.1468 - mean_absolute_percentage_erro - ETA: 1:16 - loss: 0.0395 - m

961/961 [==============================] - 100s 104ms/step - loss: 0.0382 - mean_squared_error: 0.0382 - mean_absolute_error: 0.1442 - mean_absolute_percentage_error: 147.2616 - val_loss: 0.0149 - val_mean_squared_error: 0.0149 - val_mean_absolute_error: 0.0943 - val_mean_absolute_percentage_error: 45.3598 0.1436 - mean_absolute_percentage_error: 122.16 - ETA: 1:27 - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.1436 - mean_absolute_percentage_error: 122. - ETA: 1:27 - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.1436 - mean_absolute_p - ETA: 1:23 - loss: 0.0379 - mean_squared_error: 0.0379 - mean_absolute_error: 0.1436 - mean_absolute_percentage_error:  - ETA: 1:23 - loss: 0.0379 - mean_squared_error: 0.0379 - mean_absolute_error: 0.1436 - mean_abs - ETA: 1:20 - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.1437 - mean_absolute_percentage - ETA: 1:18 - loss: 0.0380 - mean_squared_error: 0.0380 - mean_absolute_error: 0.143

481/481 [==============================] - 78s 141ms/step - loss: 0.1661 - mean_squared_error: 0.1661 - mean_absolute_error: 0.2576 - mean_absolute_percentage_error: 160.5884 - val_loss: 0.0198 - val_mean_squared_error: 0.0198 - val_mean_absolute_error: 0.0957 - val_mean_absolute_percentage_error: 36.267736 - loss: 0.8952 - mean_squared_error: 0.8952 - mean_absolute_error: 0.7126 - mean_absolute_percentage_erro - ETA: 4:50 - loss: 0.7273 - mean_squ

Epoch 00001: val_loss improved from inf to 0.01983, saving model to logs/model\LSTM_8_layers_tS_0.7_bs_128.h5
Epoch 2/30
481/481 [==============================] - 57s 119ms/step - loss: 0.0583 - mean_squared_error: 0.0583 - mean_absolute_error: 0.1675 - mean_absolute_percentage_error: 155.9579 - val_loss: 0.0161 - val_mean_squared_error: 0.0161 - val_mean_absolute_error: 0.0912 - val_mean_absolute_percentage_error: 44.1262

Epoch 00002: val_loss improved from 0.01983 to 0.01610, saving model to logs/model\LSTM_8_layers_tS_0.7_bs_128.h5
Epo

241/241 [==============================] - 55s 189ms/step - loss: 0.2353 - mean_squared_error: 0.2353 - mean_absolute_error: 0.3105 - mean_absolute_percentage_error: 174.0693 - val_loss: 0.0235 - val_mean_squared_error: 0.0235 - val_mean_absolute_error: 0.1005 - val_mean_absolute_percentage_error: 42.2642

Epoch 00001: val_loss improved from inf to 0.02349, saving model to logs/model\LSTM_8_layers_tS_0.7_bs_256.h5
Epoch 2/30
241/241 [==============================] - 36s 149ms/step - loss: 0.0667 - mean_squared_error: 0.0667 - mean_absolute_error: 0.1738 - mean_absolute_percentage_error: 151.9067 - val_loss: 0.0200 - val_mean_squared_error: 0.0200 - val_mean_absolute_error: 0.0954 - val_mean_absolute_percentage_error: 40.0194

Epoch 00002: val_loss improved from 0.02349 to 0.02001, saving model to logs/model\LSTM_8_layers_tS_0.7_bs_256.h5
Epoch 3/30
241/241 [==============================] - 36s 150ms/step - loss: 0.0602 - mean_squared_error: 0.0602 - mean_absolute_error: 0.1693 - mean

In [13]:
#!kill 15864

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
#https://stackoverflow.com/questions/57228487/valueerror-duplicate-plugins-for-name-projector
#https://github.com/tensorflow/tensorboard/issues/2483
#https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
#https://www.programcreek.com/python/example/104420/keras.callbacks.TensorBoard

%tensorboard --port 6006 --logdir logs/fit


Reusing TensorBoard on port 6003 (pid 26460), started 7 days, 22:52:07 ago. (Use '!kill 26460' to kill it.)